In [16]:
import json
import pandas as pd
from PIL import Image
from collections import defaultdict

In [ ]:
prompt_results_df = pd.read_csv('C:/Users/neals/Desktop/practicum/prompt_results_classification.csv')
prompt_results_df['prompt'] = prompt_results_df['prompt'].fillna('')
prompt_results_df.prompt.unique()
context_df = prompt_results_df[prompt_results_df['prompt']=='']
context_df.head()

In [ ]:
dataset_dir = 'E:/datasets'
dataset_name = 'DOTA_dataset_512'
coco_dir = f'{dataset_dir}/{dataset_name}'
coco_file = f'{coco_dir}/val_coco.json'
coco_file = f'{coco_dir}/YOLO_coco_results_annots.json'

with open(coco_file, 'r') as f:
    coco_json = json.load(f)

category_map = {cat['id']:cat['name'] for cat in coco_json['categories']}
annotation_df = pd.DataFrame(coco_json['annotations'])
annotation_df['category'] = annotation_df['category_id'].map(category_map)
annotation_df.head()

In [ ]:
category_agg = annotation_df.groupby('image_id')['category'].agg(lambda x: list(set(x))).reset_index()
category_agg.rename(columns={'category': 'category_list'}, inplace=True)
# Flatten the lists and find unique values
unique_values = pd.Series([item for sublist in category_agg['category_list'] for item in sublist]).unique()

unique_values

In [ ]:
context_df = context_df.merge(category_agg, on='image_id', how='left').dropna()
context_df.head(10)

In [21]:
top_ship_words = ['water','boats','dock','boat','marina','docked','ship','ocean','lake','sea']
top_plane_words =['airport','tarmac','runway','plane','planes','airplanes','airplane','military','flying','aerial']

def set_word_match_score(row, match_words):
    sentence = row.generated_text
    # Split the sentence into individual words
    words_in_sentence = set(sentence.lower().split())
    
    # Count how many words from match_words are in the sentence
    matched_words = sum(1 for word in match_words if word.lower() in words_in_sentence)
    
    # Calculate the score as matched words / total match words
    return matched_words / len(match_words)

context_df['plane_score'] = context_df.apply(lambda x: set_word_match_score(x, top_plane_words), axis=1)
context_df['ship_score'] = context_df.apply(lambda x: set_word_match_score(x, top_ship_words), axis=1)
context_df['has_plane'] = context_df['category_list'].apply(lambda x: 'plane' in x)
context_df['has_ship'] = context_df['category_list'].apply(lambda x: 'ship' in x)




In [ ]:
no_plane_context_df = context_df[(context_df['has_plane']==False)&(context_df['plane_score']>0.1)]
no_plane_context_df = no_plane_context_df.sort_values(by='plane_score',ascending=False)
no_plane_context_df.head(10)
print(len(no_plane_context_df.index))
for ind,  row in enumerate(no_plane_context_df.itertuples()):
    img = Image.open(row.image_path)
    print(row.image_path)
    print(row.plane_score)
    print(row.generated_text)
    display(img)
    if ind>5: break

In [ ]:
no_ship_context_df = context_df[(context_df['has_ship']==False)&(context_df['ship_score']>0.1)]
no_ship_context_df = no_ship_context_df.sort_values(by='ship_score',ascending=False)
no_ship_context_df.head(10)
print(len(no_ship_context_df.index))
for ind,  row in enumerate(no_ship_context_df.itertuples()):
    img = Image.open(row.image_path)
    print(row.category_list)
    print(row.image_path)
    print(row.ship_score)
    print(row.generated_text)
    display(img)
    if ind>5: break

In [ ]:
no_ship_context_df = context_df[(context_df['has_ship']==False)&(context_df['ship_score']>0.1)
                                &(~context_df['generated_text'].str.contains('dock'))
                                &(~context_df['generated_text'].str.contains('houses'))]
no_ship_context_df = no_ship_context_df.sort_values(by='ship_score',ascending=False)
no_ship_context_df.head(10)
print(len(no_ship_context_df.index))
for ind,  row in enumerate(no_ship_context_df.itertuples()):
    img = Image.open(row.image_path)
    print(row.category_list)
    print(row.image_path)
    print(row.ship_score)
    print(row.generated_text)
    display(img)
    if ind>30: break